In [1]:
import copy
import qcodes
#from qcodes import StandardParameter

In [2]:
class Powersource(qcodes.Instrument):
    def __init__(self, name): 
        super().__init__(name)
        self._voltage = 0
        
        self.add_parameter(
            name="voltage", 
            set_cmd=self._set_voltage,
            get_cmd=self._get_voltage,
            unit="mV"  # Stupid thing works in mV instead of Volt
        )
        
    def _set_voltage(self, value): 
        self._voltage = value
        
    def _get_voltage(self):
        return self._voltage

In [3]:
# This cell works, but maybe its not how we want to do things in the long run 

def alias(parameter, new_label=None, new_name=None, new_unit=None, conversion_factor=None):

    new_label = new_label or parameter.label
    new_name = new_name or parameter.name
    new_unit = new_unit or parameter.unit

    parameter_class = parameter.__class__   # Yuck! 
    
    new_parameter = parameter_class(
        name=new_name,
        instrument=parameter._instrument,   # We are accessing a private variable here, which is far from ideal
        label=new_label,
        unit=new_unit,
        set_cmd=lambda value: parameter(conversion_factor * value),
        get_cmd=lambda: parameter() / conversion_factor
    )

    return new_parameter

In [4]:
# This cell does not work (yet), but maybe this is the way forward 

def alias(parameter, mapping, **kwargs): 
    
    # The proposal is to add a copy method on the Parameter base class. The copy method shall return a new parameter 
    # of the same type and with the same attributes (including instrument) as the original, except where this is overriden 
    # by the copy arguments. 
    new_parameter = parameter.copy(
        set_cmd=lambda value: parameter(mapping(value)), 
        get_cmd=lambda: mapping.inverse(parameter()), 
        **kwargs
    )

In [4]:
pwr = Powersource("pwr")

In [5]:
gate = alias(pwr.voltage, new_label="gate voltage", new_name="gate", new_unit="V", conversion_factor=1E-3)

In [6]:
gate(3) # Set at 3 V

In [7]:
gate()

3.0

In [8]:
pwr.voltage()

0.003

In [9]:
pwr.snapshot()

{'__class__': '__main__.Powersource',
 'functions': {},
 'name': 'pwr',
 'parameters': {'IDN': {'__class__': 'qcodes.instrument.parameter.StandardParameter',
   'instrument': '__main__.Powersource',
   'instrument_name': 'pwr',
   'label': 'IDN',
   'name': 'IDN',
   'ts': None,
   'unit': '',
   'vals': '<Anything>',
   'value': None},
  'voltage': {'__class__': 'qcodes.instrument.parameter.StandardParameter',
   'instrument': '__main__.Powersource',
   'instrument_name': 'pwr',
   'label': 'voltage',
   'name': 'voltage',
   'ts': '2017-11-16 14:02:36',
   'unit': 'mV',
   'vals': '<Numbers>',
   'value': 0.003}},
 'submodules': {}}

In [10]:
pwr.voltage.snapshot()

{'__class__': 'qcodes.instrument.parameter.StandardParameter',
 'instrument': '__main__.Powersource',
 'instrument_name': 'pwr',
 'label': 'voltage',
 'name': 'voltage',
 'ts': '2017-11-16 14:02:36',
 'unit': 'mV',
 'vals': '<Numbers>',
 'value': 0.003}

In [11]:
gate.snapshot()

{'__class__': 'qcodes.instrument.parameter.StandardParameter',
 'instrument': '__main__.Powersource',
 'instrument_name': 'pwr',
 'label': 'gate voltage',
 'name': 'gate',
 'ts': '2017-11-16 14:02:36',
 'unit': 'V',
 'vals': '<Numbers>',
 'value': 3.0}

In [12]:
gate.metadata

{}

In [13]:
pwr.voltage.metadata

{}